In [1]:
import pandas as pd
import re
df=pd.read_csv("/content/drive/MyDrive/Thesis Work 2021-2022/Latest dataset/BanglaNewsCommentData.csv")

In [2]:
df.head()

,Unnamed: 0,Comment,Tag
0,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,0
1,1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,0
2,2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,0
3,3,চুরি নয় লুটপাট।,0
4,4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,2


In [3]:
df['Comment'] = df['Comment'].apply((lambda x: re.sub('[,?'']','',x)))

In [4]:
import numpy as np 
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding,SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Comment'].values)
X = tokenizer.texts_to_sequences(df['Comment'].values)
X = pad_sequences(X)

In [6]:
X

array([[   0,    0,    0, ...,   88,  302,  410],
       [   0,    0,    0, ...,   48,  961,  385],
       [   0,    0,    0, ...,  184,   50,  148],
       ...,
       [   0,    0,    0, ..., 1053,  165,  374],
       [   0,    0,    0, ...,   73,  287, 1526],
       [   0,    0,    0, ...,  406,  178,    8]], dtype=int32)

In [7]:
np.shape(X)

(13802, 90)

In [9]:
embed_dim = 64
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['acc'])

In [10]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 90, 64)            160000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 90, 64)           0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 193,219
Trainable params: 193,219
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
Y = pd.get_dummies(df['Tag']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20,random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(11041, 90) (11041, 3)
(2761, 90) (2761, 3)


In [12]:
history=model.fit(X_train, Y_train, epochs = 10, batch_size=32, verbose = 2,validation_split=0.1)


Epoch 1/10
311/311 - 109s - loss: 1.0039 - acc: 0.5285 - val_loss: 0.9703 - val_acc: 0.5674 - 109s/epoch - 352ms/step
Epoch 2/10
311/311 - 103s - loss: 0.9113 - acc: 0.5921 - val_loss: 0.9639 - val_acc: 0.5710 - 103s/epoch - 332ms/step
Epoch 3/10
311/311 - 106s - loss: 0.8467 - acc: 0.6224 - val_loss: 0.9823 - val_acc: 0.5557 - 106s/epoch - 342ms/step
Epoch 4/10
311/311 - 104s - loss: 0.8038 - acc: 0.6452 - val_loss: 1.0071 - val_acc: 0.5575 - 104s/epoch - 333ms/step
Epoch 5/10
311/311 - 102s - loss: 0.7586 - acc: 0.6701 - val_loss: 1.0500 - val_acc: 0.5412 - 102s/epoch - 328ms/step
Epoch 6/10
311/311 - 104s - loss: 0.7209 - acc: 0.6888 - val_loss: 1.0594 - val_acc: 0.5475 - 104s/epoch - 335ms/step
Epoch 7/10
311/311 - 103s - loss: 0.6815 - acc: 0.7075 - val_loss: 1.1126 - val_acc: 0.5276 - 103s/epoch - 331ms/step
Epoch 8/10
311/311 - 104s - loss: 0.6506 - acc: 0.7288 - val_loss: 1.1576 - val_acc: 0.5267 - 104s/epoch - 335ms/step
Epoch 9/10
311/311 - 103s - loss: 0.6158 - acc: 0.7400 -

In [13]:
score = model.evaluate(X_train, Y_train,
                       batch_size=64, verbose=2)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

173/173 - 4s - loss: 0.5445 - acc: 0.7912 - 4s/epoch - 23ms/step
Train loss: 0.544456958770752
Train accuracy: 0.791232705116272


In [14]:
score = model.evaluate(X_test, Y_test,
                       batch_size=64, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

44/44 - 1s - loss: 1.3191 - acc: 0.4995 - 1s/epoch - 23ms/step
Test loss: 1.319128155708313
Test accuracy: 0.4994567334651947


In [15]:
y_preds=model.predict(X_test)

87/87 [==============================] - 2s 24ms/step


In [16]:
y_preds

array([[0.18688245, 0.5713605 , 0.24175698],
       [0.28374395, 0.4034243 , 0.31283173],
       [0.8338505 , 0.01078724, 0.15536223],
       ...,
       [0.7781021 , 0.1832564 , 0.03864157],
       [0.6328395 , 0.35768396, 0.00947649],
       [0.89606446, 0.01030488, 0.09363068]], dtype=float32)

In [19]:
y_test=Y_test.flatten()

In [20]:
y_test

array([1, 0, 0, ..., 1, 0, 0], dtype=uint8)

In [21]:
y_pred = y_preds.flatten()

In [22]:
y_pred 

array([0.18688245, 0.5713605 , 0.24175698, ..., 0.89606446, 0.01030488,
       0.09363068], dtype=float32)

In [23]:
y_predicted = np.where(y_pred > 0.5, 1, 0)
y_predicted

array([0, 1, 0, ..., 1, 0, 0])

In [24]:
import sklearn.metrics

In [26]:
r = sklearn.metrics.confusion_matrix(y_test, y_predicted)
print(r)

[[4364 1158]
 [1526 1235]]


In [28]:
r = sklearn.metrics.classification_report(y_test, y_predicted)
print(r)

              precision    recall  f1-score   support

           0       0.74      0.79      0.76      5522
           1       0.52      0.45      0.48      2761

    accuracy                           0.68      8283
   macro avg       0.63      0.62      0.62      8283
weighted avg       0.67      0.68      0.67      8283

